# MNIST: fully connected network



In [21]:
import numpy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt

SEED = 1

# CUDA?
cuda = torch.cuda.is_available()

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

In [22]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

print (train)
# Create DataLoader
dataloader_args = dict(shuffle=True, batch_size=256,num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)


train_data = train.transform(train.train_data.numpy())


Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./data
    Transforms (if any): Compose(
                             ToTensor()
                         )
    Target Transforms (if any): None


In [23]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [24]:
# One hidden Layer NN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Linear(784, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        x = x.view((-1, 784))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        return F.log_softmax(h, dim=0)    
    
    
model = Model()
if cuda:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [25]:
#training 
EPOCHS = 5
losses = []

# Eval
evaluate_x = Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.test_labels)

train_size = len(train_loader.dataset)
batch_size = (train_size / 256) if (cuda) else  (train_size / 64)

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        data, target = Variable(data), Variable(target)
    
        # Init
        optimizer.zero_grad()

        # Predict
        y_pred = model(data) 
         
        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.cpu().item())
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        # Display
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                EPOCHS,
                batch_idx * len(data), 
                train_size,
                100. * batch_idx / batch_size, 
                loss.cpu().item()), 
                end='')
    # display final evaluation for this epoch
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    d = pred.eq(evaluate_y.data).cpu()
    accuracy = d.sum().item()/d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.4f}%'.format(
        epoch+1,
        EPOCHS,
        train_size, 
        train_size,
        100. * batch_idx / batch_size, 
        loss.cpu().item(),
        accuracy*100,
        end=''))

 Train Epoch: 1/5 [60000/60000 (100%)]	Loss: 0.087640	 Test Accuracy: 96.5500%
 Train Epoch: 2/5 [60000/60000 (100%)]	Loss: 0.160783	 Test Accuracy: 96.9000%
 Train Epoch: 3/5 [60000/60000 (100%)]	Loss: 0.216033	 Test Accuracy: 97.4100%
 Train Epoch: 4/5 [60000/60000 (100%)]	Loss: 0.082945	 Test Accuracy: 97.6400%
 Train Epoch: 5/5 [60000/60000 (100%)]	Loss: 0.008376	 Test Accuracy: 98.2000%


In [33]:
def test(test_loader,model,eps):
    #Adversarial images
    adv_im = []
    for data, target in test_loader:
        
        # Get Samples        
        data, target = Variable(data), Variable(target) 
        data.requires_grad = True
        output = model(data)
        pred = output.data.max(1)[1]

        # Calculate loss
        loss = F.cross_entropy(output, target)
        
                
        # Init
        model.zero_grad()       

        # Backpropagation
        loss.backward()
        
        data_grad = data.grad.data
        
        #perturb the data 
        perturbed_data = fgsm_attack(data, eps, data_grad)
        
        #Classify the perturbed data
        output = model(perturbed_data)
        
        #Calculate the accuracy
        final_pred = output.data.max(1)[1]
        d = final_pred.eq(target.data).cpu()        
        accuracy = d.sum().item()/d.size()[0]
        
        
        if len(adv_im) < 5:
            adv_ex = perturbed_data.squeeze().detach().cpu().numpy()   
            adv_im.append( (pred[1], final_pred[1], adv_ex) ) 
    return accuracy,adv_im

In [37]:
accuracies = []
examples = []
epsilons = [0, .1, .15, .2, .25, .3]
# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(test_loader,model, eps)
    accuracies.append(acc)
    examples.append(ex)

In [39]:
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv))
        plt.imshow(ex[1], cmap="gray")
plt.tight_layout()
plt.show()
